In [42]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json

In [43]:
device = torch.device("cuda")

In [20]:
# 일단 데이터
data = [
    {
        "id": 1,
        "content": "파란 하늘 아래에서 사랑하는 사람들과 함께 손을 잡고, 웃음이 넘쳐나는 순간들. 모든 게 완벽하게 맞아떨어지는 이 순간, 나의 마음은 행복으로 가득 차 있다. 내 마음 속에서 기쁨이 춤추고 있다."
    },
    {
        "id": 2,
        "content": "오늘 너무 행복해서 날아갈 것만 같고 너무 기쁘고 기쁘고 기쁘고 정말 기쁘다."
    },
    {
        "id": 3,
        "content": "햇살 가득한 정원에서 친구들과 웃으며 피크닉을 즐기고 있어요. 모든 걱정을 잊고, 그냥 지금 이 순간에만 집중하는 기분이에요. 행복이 내 몸속 깊숙이 퍼지고 있어요!"
    },
    {
        "id": 4,
        "content": "산길을 걷던 남자는 우연히 떨어진 그림엽서를 발견했다. 그림 속에는 자신이 어릴 적 갔던 바다가 그려져 있었다. 뒷면에는 낯익은 필체로 '다시 만나자'라는 문구가 적혀 있었다. 누군가의 초대 같았다."
    },
    {
        "id": 5,
        "content": "한밤중에 잠에서 깬 소년은 창문 밖에서 들려오는 이상한 소리를 들었다. 밖을 내다보니 작은 고양이가 소리 내어 울고 있었다. 소년은 고양이를 집 안으로 데려왔고, 그날 이후 둘은 떨어질 수 없는 친구가 되었다."
    },
    {
        "id": 6,
        "content": "강가에서 낚시를 하던 노인은 물속에서 빛나는 병을 발견했다. 병 속에는 오래된 쪽지가 들어 있었는데, '이 병을 발견한 이는 행운을 얻을 것이다'라고 적혀 있었다. 그날부터 노인의 삶은 놀랍도록 변하기 시작했다."
    },
    {
        "id": 7,
        "content": "비 오는 날, 한 소녀는 학교로 가는 길에 낡은 우산을 발견했다. 우산에는 '행운의 우산'이라는 작은 메모가 붙어 있었다. 소녀는 우산을 쓰기 시작했고, 이상하게도 그날 이후 좋은 일이 계속해서 일어났다."
    },
    {
        "id": 8,
        "content": "숲속 깊은 곳에 있는 작은 오두막에는 매일 밤 불빛이 켜졌다. 마을 사람들은 그곳에 누군가가 산다는 소문을 들었지만, 아무도 그를 본 적이 없었다. 어느 날, 용기를 낸 한 소년이 오두막을 찾아갔고 놀라운 비밀을 발견했다."
    },
    {
        "id": 9,
        "content": "늦은 밤, 정류장에서 버스를 기다리던 남자는 이상한 기운을 느꼈다. 누군가 자신을 지켜보는 것 같았다. 고개를 돌리자, 낡은 벤치 위에 오래된 사진이 놓여 있었다. 사진 속에는 그와 똑같이 생긴 사람이 서 있었다."
    },
    {
        "id": 10,
        "content": "한적한 해변에서 발견된 오래된 일기장은 누군가의 비밀로 가득 차 있었다. 글을 읽던 소녀는 일기 속 주인공과 자신의 삶이 이상하리만큼 닮아 있다는 것을 알게 되었다. 마치 자신을 위한 이야기처럼 느껴졌다."
    },
    {
        "id": 11,
        "content": "노을이 지는 강가에서 한 남자가 낚시를 하고 있었다. 물속에서 무언가 빛나는 것을 발견한 그는 손을 뻗어 그것을 꺼냈다. 그것은 오래된 황금 반지였다. 반지에는 '영원한 사랑'이라는 글자가 새겨져 있었고, 그 후로 그의 삶은 놀라운 방향으로 흘러갔다."
    },
    {
        "id": 12,
        "content": "작은 도서관의 구석에서 낡은 책 한 권이 발견되었다. 책의 첫 페이지에는 '읽는 이에게 행운을'이라는 문구가 적혀 있었다. 책을 읽기 시작한 소녀는 매일 꿈속에서 책 속의 이야기를 경험하며 놀라운 모험을 시작했다."
    },
    {
        "id": 13,
        "content": "도시의 오래된 골목길에는 늘 기타를 치는 한 소년이 있었다. 그는 누구도 모르는 곡을 연주했지만, 그 음악은 지나가는 사람들의 마음을 울렸다. 어느 날, 한 청중이 다가와 '이 곡은 어디서 배운 거니?'라고 묻자 소년은 '꿈에서 들었다'고 답했다."
    },
    {
        "id": 14,
        "content": "버려진 놀이터에서 놀던 아이들은 지하에 숨겨진 비밀 문을 발견했다. 문을 열자 거대한 미로가 나타났고, 아이들은 미로 속에서 환상적인 모험을 시작했다. 그곳에는 꿈에서만 보던 동물들과 마법의 세계가 기다리고 있었다."
    },
    {
        "id": 15,
        "content": "작은 카페의 메뉴판 뒤에는 오래된 편지가 숨겨져 있었다. 한 여성이 우연히 발견한 편지에는 '사랑하는 이에게'라는 제목과 함께 과거의 가슴 절절한 이야기가 담겨 있었다. 그녀는 편지의 주인을 찾아 나섰고, 놀라운 인연을 만나게 되었다."
    },
    {
        "id": 16,
        "content": "학교 뒷마당에서 놀던 소년은 땅에 묻힌 작은 상자를 발견했다. 상자 속에는 오래된 동전과 함께 '용기를 가지면 모든 것을 얻을 수 있다'는 글귀가 적힌 쪽지가 있었다. 소년은 이후 도전하는 삶을 살며 꿈을 이루기 시작했다."
    },
    {
        "id": 17,
        "content": "한밤중에 잠에서 깬 한 남자는 창밖에 빛나는 별 하나가 점점 커지는 것을 보았다. 별은 그의 방으로 들어와 작은 구슬이 되었고, 구슬은 그에게 소원을 들어줄 수 있다고 말했다. 그는 인생의 가장 중요한 소원을 빌기로 결심했다."
    },
    {
        "id": 18,
        "content": "낯선 도시의 오래된 다리 위에서 한 소녀가 사진을 찍고 있었다. 사진 속에 나타난 희미한 그림자는 그녀가 잃어버린 친구를 닮아 있었다. 소녀는 진실을 찾기 위해 다리 아래 숨겨진 비밀을 조사하기 시작했다."
    },
    {
        "id": 19,
        "content": "가을의 어느 날, 낡은 나무 아래에서 한 청년이 오래된 손수건을 발견했다. 손수건에는 정성스럽게 수놓아진 이니셜과 함께 '기억의 조각'이라는 글자가 새겨져 있었다. 그는 손수건의 주인을 찾기 위해 여정을 떠났다."
    },
    {
        "id": 20,
        "content": "한 마을에는 전설이 있었다. 마을 중앙의 큰 나무에 걸린 종을 울리면 소원을 이룰 수 있다는 것이다. 어느 날, 한 아이가 용기를 내어 종을 울렸다. 하지만 종소리와 함께 나타난 것은 소원이 아닌 마을의 오래된 비밀이었다."
    },
]

In [21]:
# 1. 데이터 로드
train = pd.read_csv('../datasets/sentiment_conversation/train.csv')
val = pd.read_csv('../datasets/sentiment_conversation/val.csv')

In [22]:
print(train["sentiment"].value_counts())

sentiment
4    26301
2    26000
3    25957
1    25814
5    24936
0    16947
Name: count, dtype: int64


In [23]:
print(train.head())

   sentiment                                               text
0          2                          일은 왜 해도 해도 끝이 없을까? 화가 난다.
1          2     이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2          2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3          2  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4          2              얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [24]:
print(val.head())

   sentiment                                               text
0          4  이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1          4              회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.
2          4                      상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.
3          4          이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.
4          4                       직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.


In [25]:
# 2. KLUE/roberta-base 토크나이저 적용
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at C:\Users\kimso/.cache\huggingface\transformers\e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at C:\Users\kimso/.cache\huggingface\transformers\233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at C:\Users\kimso/.cache\huggingface\transformers\9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokeni

In [41]:
# 3. Hugging Face Dataset 변환
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])

train_dataset = train_dataset.rename_column("sentiment", "labels")
val_dataset = val_dataset.rename_column("sentiment", "labels")

train_dataset.set_format("torch")
val_dataset.set_format("torch")

# 추가
small_train_dataset = train_dataset.select(range(100))  # 100개만 학습
small_val_dataset = val_dataset.select(range(20))  # 20개만 평가





  0%|          | 360/91230 [07:23<31:05:32,  1.23s/it]






















Map:  49%|████▉     | 72000/145955 [00:03<00:04, 18204.69 examples/s]


KeyboardInterrupt: 

In [ ]:
# 4. 모델 정의
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(train["sentiment"].unique()))
model.to(device)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at C:\Users\kimso/.cache\huggingface\transformers\a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
 

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [39]:
# 5. TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [40]:
# 6. Trainer 정의 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,  # 소규모 데이터셋 사용
    eval_dataset=small_val_dataset,  # 소규모 검증 데이터 사용
)

***** Running training *****
  Num examples = 145955
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27369
                                                     
  0%|          | 360/91230 [02:39<4:25:02,  5.71it/s]


{'loss': 1.7838, 'learning_rate': 4.908655778435456e-05, 'epoch': 0.05}


                                                     
  0%|          | 360/91230 [04:07<4:25:02,  5.71it/s] 

{'loss': 1.7939, 'learning_rate': 4.8173115568709124e-05, 'epoch': 0.11}


                                                     
  0%|          | 360/91230 [05:35<4:25:02,  5.71it/s] 

{'loss': 1.7907, 'learning_rate': 4.7259673353063685e-05, 'epoch': 0.16}


KeyboardInterrupt: 

In [ ]:
trainer.train()

In [36]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [37]:
for item in data:
    inputs = tokenizer(item['content'], return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.softmax(outputs.logits, dim=1)  # 감성 확률 계산
        predicted_label = torch.argmax(scores, dim=1).item()

    print(f"Text: {item['content']}")
    print(f"Scores: {scores.tolist()}")
    print(f"Predicted label: {predicted_label}\n")

Text: 파란 하늘 아래에서 사랑하는 사람들과 함께 손을 잡고, 웃음이 넘쳐나는 순간들. 모든 게 완벽하게 맞아떨어지는 이 순간, 나의 마음은 행복으로 가득 차 있다. 내 마음 속에서 기쁨이 춤추고 있다.
Scores: [[0.20696835219860077, 0.16610361635684967, 0.17177818715572357, 0.1654752939939499, 0.1356823444366455, 0.1539921909570694]]
Predicted label: 0

Text: 오늘 너무 행복해서 날아갈 것만 같고 너무 기쁘고 기쁘고 기쁘고 정말 기쁘다.
Scores: [[0.210343137383461, 0.16448689997196198, 0.16351763904094696, 0.16429173946380615, 0.14139224588871002, 0.1559683233499527]]
Predicted label: 0

Text: 햇살 가득한 정원에서 친구들과 웃으며 피크닉을 즐기고 있어요. 모든 걱정을 잊고, 그냥 지금 이 순간에만 집중하는 기분이에요. 행복이 내 몸속 깊숙이 퍼지고 있어요!
Scores: [[0.2131807953119278, 0.16164669394493103, 0.1680438071489334, 0.16505427658557892, 0.1355336606502533, 0.15654070675373077]]
Predicted label: 0

Text: 산길을 걷던 남자는 우연히 떨어진 그림엽서를 발견했다. 그림 속에는 자신이 어릴 적 갔던 바다가 그려져 있었다. 뒷면에는 낯익은 필체로 '다시 만나자'라는 문구가 적혀 있었다. 누군가의 초대 같았다.
Scores: [[0.2065226286649704, 0.16660113632678986, 0.16835784912109375, 0.16338993608951569, 0.13892100751399994, 0.15620744228363037]]
Predicted label: 0


In [15]:
# 결과 출력
output_file = './results/using_test_data.json'
with open(output_file, 'w', encoding='utf-8') as json_f:
    json.dump(data, json_f, ensure_ascii=False, indent=4)